# 简介
你只需要按照这份教程一步一步操作，即可实现一个「自定义知识库的聊天机器人」。 本教程的使用方法很简单，你只需要点击代码前的播放按钮即可。

# 安装依赖库

点击下方代码前的播放按钮即可，这一步主要是安装好后续程序依赖的库。

In [ ]:
!pip install gpt-index
!pip install langchain
!pip install transformers

# 导入个人知识库数据

1. 左侧文件夹菜单中新建 `data` 文件夹。

2. 上传知识库文件到文件夹内。

# 设置 OpenAI API Key

点击代码前的播放按钮，运行代码后，在框内输入你的个人 OpenAI API Key，黏贴完记得按下回车键。如何生成 API Key 请见[教程]()。

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = input("将你的 OpenAI API Key 黏贴到这里：")

# 定义函数

## 构建索引 
`construct_index(path)`

这个方法用 OpenAI 的 embedding API 来构建索引，这个 API 可以将文本转换为向量，这样就可以通过计算向量的相似度来判断两个文本的相似度。


## 查询索引函数 

`ask_ai()`

这个方法用来查询索引，输入问题，返回答案。

In [ ]:
from gpt_index import SimpleDirectoryReader, GPTListIndex, readers, GPTSimpleVectorIndex, LLMPredictor, PromptHelper
from langchain import OpenAI
import sys
from IPython.display import Markdown, display

def construct_index(directory_path):
    # set maximum input size
    max_input_size = 4096
    # set number of output tokens
    num_outputs = 300
    # set maximum chunk overlap
    max_chunk_overlap = 20
    # set chunk size limit
    chunk_size_limit = 600 

    # define LLM
    llm_predictor = LLMPredictor(llm=OpenAI(temperature=0.5, model_name="text-davinci-003", max_tokens=num_outputs))
    prompt_helper = PromptHelper(max_input_size, num_outputs, max_chunk_overlap, chunk_size_limit=chunk_size_limit)
 
    documents = SimpleDirectoryReader(directory_path).load_data()
    
    index = GPTSimpleVectorIndex(
        documents, llm_predictor=llm_predictor, prompt_helper=prompt_helper
    )

    index.save_to_disk('index.json')

    return index

def ask_ai():
    index = GPTSimpleVectorIndex.load_from_disk('index.json')
    while True: 
        query = input("请输入你的提问：")
        response = index.query(query, response_mode="default")
        display(Markdown(f"Response: <b>{response.response}</b>"))

# 构建索引
这一步程序会将第一步导入的数据都跑一遍，并使用 OpenAI 的 embedings API 去生成向量数据索引

In [ ]:
construct_index('data')

# 提问

这一步你就可以试试提问了

- Viberate 的范围是多少？
- Pump 的最大值是多少？

In [ ]:
ask_ai()